<a href="https://colab.research.google.com/github/webb-e/S2_Landsat_Comparison/blob/main/CloudOccurrence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
author: @ericslevenson
date: Nov 6, 2022; updated Jan 23, 2025 by Elizabeth Webb
description: Ingest map of buffered lakes, roi, timeframe. Export annual cloud and surface water occurrence map
Needs to be run seperately for each region in each year
'''

In [ ]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
from google.colab import auth
auth.authenticate_user()

# Earth Engine setup
import ee # Trigger the authentication flow.
ee.Authenticate()
ee.Initialize(project=" ") # Initialize the library.

# Some common imports
from IPython.display import Image
import folium

In [ ]:
## ***INPUTS***


## Regularly Adjusted Inputs ##
year = str(2016)
id = '08WNA'

start = year + '-05-01'
finish = year + '-09-30'

# Export settings
description = str(id) + '_TUKMRD_' + year  # filename
#description = 'TUKMRD_07WFT_2020' # filename

## Periodically Adjusted Inputs ##
# Lake Shapefile
lakes = ee.FeatureCollection('projects/ee-webbe/assets/ALPOD/Tuk_MRD_unet_shps')
# Tiled ROI
tiles = ee.FeatureCollection('projects/ee-webbe/assets/ALPOD/S2_tiles')

## Rarely Adjusted Input ##
# Image scale
pixScale = 10

## ***EARTH ENGINE-IFY***
eeroi = tiles.filter(ee.Filter.eq('Name', id)).first() # Filter grid to tile to define roi
#eeroi = ee.FeatureCollection('projects/ee-webbe/assets/ALPOD/TUK_roi')
roi = ee.Geometry.Polygon(eeroi.geometry().getInfo()['coordinates'][0]) # define roi as geometry variable
#roi = ee.Geometry.MultiPolygon(eeroi.geometry().getInfo()['coordinates']) #
startDoy = ee.Date(start).getRelative('day', 'year')
endDoy = ee.Date(finish).getRelative('day', 'year')
eestart = ee.Date(start)
eefinish = ee.Date(finish)



### Functions

In [ ]:
## ***IMAGE PRE-PROCESSING METHODS***

# Mask clouds in Sentinel-2
def maskS2clouds(image):
  '''Takes an input and adds two bands: cloud mask and clear mask'''
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  clear_mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).rename('clear_mask')
  cloud_mask = qa.bitwiseAnd(cloudBitMask).eq(1).And(qa.bitwiseAnd(cirrusBitMask).eq(1)).rename('cloud_mask')
  return image.addBands([cloud_mask,clear_mask])

# Clip image
def clip_image(image):
  '''Clips to the roi defined at the beginning of the script'''
  return image.clip(roi)

def clip2lakes(image):
  '''Clips an image based on the lake boundaries'''
  return image.clip(lakes)

# Get percentile cover - saved as metadata
def getCover(image):
  '''calculates percentage of the roi covered by the clear mask. NOTE: this function
  calls the global totPixels variable that needs to be calculated in the main script.'''
  actPixels = ee.Number(image.updateMask(image.select('clear_mask')).reduceRegion(
      reducer = ee.Reducer.count(),
      scale = 100,
      geometry = roi,
      maxPixels=1e12,
      ).values().get(0))
  # calculate the perc of cover OF CLEAR PIXELS
  percCover = actPixels.divide(totPixels).multiply(100).round()
  ice = image.select('B4').gte(1050)
  maskedice = image.mask(ice.Not())
  icePixels = ee.Number(maskedice
          .reduceRegion(
                reducer = ee.Reducer.count(),
                scale = 100,
                geometry = roi,
                maxPixels = 1e12,
                ).values().get(0))
  percIce = icePixels.divide(totPixels).multiply(100).round()
  # number as output
  return image.set('percCover', percCover,'actPixels',actPixels, 'ice_free_cover', percIce)

  #add count to metadata
def countmeta(image):
    counted =  ee.Number(image.updateMask(image.select('clear_mask')).reduceRegion(
        reducer =  ee.Reducer.count(),
        geometry = roi,
        scale = 100,
        maxPixels=1e12).values().get(0))
    return image.set('countval', counted)


  # Get cloud cover - saved as raster
def add_S2_footprint(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  clmask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  FPmask = image.mask().select([0]).rename('FPmask')
  return image.updateMask(clmask).addBands(FPmask)

def get_cloudiness(image):
    masky = image.mask().add(image.select('FPmask').eq(0)).eq(0)
    return masky.clip(roi)

# ice mask
def icemask(image):
    ice = image.select('B4').gte(1050)
    maskedice = image.mask(ice.Not())
    actPixels = ee.Number(maskedice
          .reduceRegion(
                reducer = ee.Reducer.count(),
                scale = 100,
                geometry = roi,
                maxPixels = 1e12,
                ).values().get(0))
    percCover = actPixels.divide(totPixels).multiply(100).round()
   # return maskedice.set('ice_free_cover', percCover)
    return maskedice

# Mosaic images by date, orbit, - basically combines images together that were taken on the same day
def mosaicBy(imcol):
  '''Takes an image collection (imcol) and creates a mosaic for each day
  Returns: An image collection of daily mosaics'''
  #return the collection as a list of images (not an image collection)
  imlist = imcol.toList(imcol.size())
  # Get all the dates as list
  def imdate(im):
    date = ee.Image(im).date().format("YYYY-MM-dd")
    return date
  all_dates = imlist.map(imdate)
  # get all orbits as list
  def orbitId(im):
    orb = ee.Image(im).get('SENSING_ORBIT_NUMBER')
    return orb
  all_orbits = imlist.map(orbitId)
  # get all spacecraft names as list
  def spacecraft(im):
    return ee.Image(im).get('SPACECRAFT_NAME')
  all_spNames = imlist.map(spacecraft)
  # this puts dates, orbits and names into a nested list
  concat_all = all_dates.zip(all_orbits).zip(all_spNames);
  # here we unnest the list with flatten, and then concatenate the list elements with " "
  def concat(el):
    return ee.List(el).flatten().join(" ")
  concat_all = concat_all.map(concat)
  # here, just get distinct combintations of date, orbit and name
  concat_unique = concat_all.distinct()
  # mosaic
  def mosaicIms(d):
    d1 = ee.String(d).split(" ")
    date1 = ee.Date(d1.get(0))
    orbit = ee.Number.parse(d1.get(1)).toInt()
    spName = ee.String(d1.get(2))
    im = imcol.filterDate(date1, date1.advance(1, "day")).filterMetadata('SPACECRAFT_NAME', 'equals', spName).filterMetadata('SENSING_ORBIT_NUMBER','equals', orbit).mosaic()
    return im.set(
        "system:time_start", date1.millis(),
        "system:date", date1.format("YYYY-MM-dd"),
        "system:id", d1)
  mosaic_imlist = concat_unique.map(mosaicIms)
  return ee.ImageCollection(mosaic_imlist)

def reprojectMosaic(image):
  '''Reproject to UTM. A future function should take the image location and return
  the UTM zone. For now, I'm manually entering the EPSG code.'''
  image_projected = image.reproject(epsg)
  return image_projected

###########################################################################
## ***WATER CLASSIFICATION METHODS***

# Define NDWI image
def ndwi(image):
  '''Adds an NDWI band to the input image'''
  return image.normalizedDifference(['B3', 'B8']).rename('NDWI').multiply(1000)

# Basic ndwi classification
def ndwi_classify(image):
  '''Creates a binary image based on an NDWI threshold of 0'''
  ndwimask = image.select('NDWI')
  water = ndwimask.gte(0)
  land = ndwimask.lt(0)
  return(water)

# OTSU thresholding from histogram
def otsu(histogram):
  '''Returns the NDWI threshold for binary water classification'''
  counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
  means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
  size = means.length().get([0])
  total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
  sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
  mean = sum.divide(total)
  indices = ee.List.sequence(1, size)
  def func_xxx(i):
    '''Compute between sum of squares, where each mean partitions the data.'''
    aCounts = counts.slice(0, 0, i)
    aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0, 0, i)
    aMean = aMeans.multiply(aCounts) \
        .reduce(ee.Reducer.sum(), [0]).get([0]) \
        .divide(aCount)
    bCount = total.subtract(aCount)
    bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
    return aCount.multiply(aMean.subtract(mean).pow(2)).add(
           bCount.multiply(bMean.subtract(mean).pow(2)))
  bss = indices.map(func_xxx)
  # Return the mean value corresponding to the maximum BSS.
  return means.sort(bss).get([-1])

# OTSU thresholding for an image
def otsu_thresh(water_image):
  '''Calculate NDWI and create histogram. Return the OTSU threshold.'''
  NDWI = ndwi(water_image).select('NDWI').updateMask(water_image.select('clear_mask'))
  histogram = ee.Dictionary(NDWI.reduceRegion(
    geometry = roi,
    reducer = ee.Reducer.histogram(255, 2).combine('mean', None, True).combine('variance', None, True),
    scale = pixScale,
    maxPixels = 1e12
  ))
  return otsu(histogram.get('NDWI_histogram'))

# Classify an image using OTSU threshold.
def otsu_classify(water_image):
  '''(1) Calculate NDWI and create histogram. (2) Calculate NDWI threshold for
  binary classification using OTSU method. (3) Classify image and add layer to input image.
  '''
  NDWI = ndwi(water_image).select('NDWI')
  histogram = ee.Dictionary(NDWI.reduceRegion(
    geometry = roi,
    reducer = ee.Reducer.histogram(255, 2).combine('mean', None, True).combine('variance', None, True),
    scale = pixScale,
    maxPixels = 1e12
  ))
  threshold = otsu(histogram.get('NDWI_histogram'))
  otsu_classed = NDWI.gt(ee.Number(threshold)).And(water_image.select('B8').lt(2000)).rename('otsu_classed')
  return water_image.addBands([otsu_classed])

def adaptive_thresholding(water_image):
  '''Takes an image clipped to lakes and returns the water mask'''
  NDWI = ndwi(water_image).select('NDWI')#.updateMask(water_image.select('clear_mask')) # get NDWI **TURNED OFF CLOUD MASK, SHOULD THIS STAY OFF?**
  threshold = ee.Number(otsu_thresh(water_image))
  threshold = threshold.divide(10).round().multiply(10)
  # get fixed histogram
  histo = NDWI.reduceRegion(
      geometry = roi,
      reducer = ee.Reducer.fixedHistogram(-1000, 1000, 200),
      scale = pixScale, # This was 30, keep at 10!?!?
      maxPixels = 1e12
  )
  hist = ee.Array(histo.get('NDWI'))
  counts = hist.cut([-1,1])
  buckets = hist.cut([-1,0])
  #find split points from otsu threshold
  threshold = ee.Array([threshold]).toList()
  buckets_list = buckets.toList()
  split = buckets_list.indexOf(threshold)
  # split into land and water slices
  land_slice = counts.slice(0,0,split)
  water_slice = counts.slice(0,split.add(1),-1)
  # find max of land and water slices
  land_max = land_slice.reduce(ee.Reducer.max(),[0])
  water_max = water_slice.reduce(ee.Reducer.max(),[0])
  land_max = land_max.toList().get(0)
  water_max = water_max.toList().get(0)
  land_max = ee.List(land_max).getNumber(0)
  water_max = ee.List(water_max).getNumber(0)
  #find difference between land, water and otsu val
  counts_list = counts.toList()
  otsu_val = ee.Number(counts_list.get(split))
  otsu_val = ee.List(otsu_val).getNumber(0)
  land_prom = ee.Number(land_max).subtract(otsu_val)
  water_prom = ee.Number(water_max).subtract(otsu_val)
  #find land and water buckets corresponding to 0.9 times the prominence
  land_thresh = ee.Number(land_max).subtract((land_prom).multiply(ee.Number(0.9)))
  water_thresh = ee.Number(water_max).subtract((water_prom).multiply(ee.Number(0.9)))
  land_max_ind = land_slice.argmax().get(0)
  water_max_ind = water_slice.argmax().get(0)
  li = ee.Number(land_max_ind).subtract(1)
  li = li.max(ee.Number(1))
  wi = ee.Number(water_max_ind).add(1)
  wi = wi.min(ee.Number(199))
  land_slice2 = land_slice.slice(0,li,-1).subtract(land_thresh)
  water_slice2 = water_slice.slice(0,0,wi).subtract(water_thresh)
  land_slice2  = land_slice2.abs().multiply(-1)
  water_slice2 = water_slice2.abs().multiply(-1)
  land_index = ee.Number(land_slice2.argmax().get(0)).add(land_max_ind)
  water_index = ee.Number(water_slice2.argmax().get(0)).add(split)
  land_level = ee.Number(buckets_list.get(land_index))
  water_level = ee.Number(buckets_list.get(water_index))
  land_level = ee.Number(ee.List(land_level).get(0)).add(5)
  water_level = ee.Number(ee.List(water_level).get(0)).add(5)
  #calculate water fraction and classify
  water_fraction = (NDWI.subtract(land_level)).divide(water_level.subtract(land_level)).multiply(100).rename('water_fraction')
  #water_fraction = conditional(water_fraction) #sets values less than 0 to 0 and greater than 100 to 100
  water_75 = water_fraction.gte(75).rename('water_75'); #note, this is a non-binary classification, so we use 75% water as "water"
  all_mask = water_image.select('B2').gt(5).rename('all_mask')
  cloud_mask_ed = water_image.select('clear_mask').neq(1).rename('cloud_mask_ed')
  return water_image.addBands([water_fraction,water_75,NDWI,cloud_mask_ed])
# Apply cloud mask to other bands
def applyMask(image):
  img = image.updateMask(image.select('clear_mask'))
  return img
def binaryImage(image):
  '''takes a multiband image and returns just the binary water_75 band'''
  img = image.select('water_75')
  return img
def waterImage(image):
  '''takes a multiband image and returns just the water fraction band'''
  img = image.select('water_fraction')
  return img
###############################################################################


### Main

In [ ]:
## ***Lake Occurrence Raster***

# (1) image pre-process:

# Get images and filter images by cloudiness, roi, time period, and month range
images = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi).filterDate(start,finish).filter(ee.Filter.calendarRange(startDoy, endDoy, 'day_of_year')).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50))
# Mosaic images and add cloud/clear masks
images_all = mosaicBy(images)
images_all = images_all.map(maskS2clouds)
# Clip mosaics to roi
images_all = images_all.map(clip_image)##.map(countmeta).filterMetadata('countval','not_equals', None)##.filter(ee.Filter.notNull(['countval']))##..filterMetadata('countval','not_equals', NaN)

# Get percent cover for each mosaic
image_mask = images.select('B2').mean().clip(roi).gte(0) #First, calculate total number of pixels
totPixels = ee.Number(image_mask.reduceRegion(
    reducer = ee.Reducer.count(),
    scale = 100,
    geometry = roi,
    maxPixels = 1e12
    ).values().get(0))
images_all = images_all.map(getCover)
# Filter by percent cover
images = images_all.filterMetadata('percCover','greater_than',70).filterMetadata('ice_free_cover','greater_than',70)
 # remove images covering less than 70% of the ROI)
# Clip remaining mosaics to buffered lake shapefile
lakeimages = images.map(clip2lakes) # Clip images to buffered lake mask

# (2) water classification:

# Apply cloud mask to other bands
lakeimages = lakeimages.map(applyMask)
# Apply ice mask
lakeimages = lakeimages.map(icemask) # remove images w/ all ice/snow

# adaptive thresholding on every lake image
lakeimages = lakeimages.map(adaptive_thresholding)
# create image collection of just binary water images
lakeimages = lakeimages.map(binaryImage)

# (3) Lake occurrence for entire date range

# reduce image collection to mean value
lakeimage = lakeimages.reduce(ee.Reducer.mean()).rename('water_occurance') # this accounts for masked pixels

# (4) Number of images used to determine occurance
obsimage = lakeimages.reduce(ee.Reducer.count()).rename('num_observations')

# (5) Cloudiness -- help from https://gis.stackexchange.com/questions/420869/reducing-imagecollection-and-calculating-percentage-of-masked-pixels-in-gee

c_images = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi).filterDate(start,finish).filter(ee.Filter.calendarRange(startDoy, endDoy, 'day_of_year'))

cloudimages = c_images.map(add_S2_footprint)
cloudims = cloudimages.map(get_cloudiness)
cloudimage = cloudims.sum().divide(cloudimages.select('FPmask').sum()).select('B1').rename('cloud_occurance');


# (6) Water + cloud occurance + number of observations
finalimage = lakeimage.addBands(cloudimage).addBands(obsimage)

# (7) Export lake occurrence mask to drive
task = ee.batch.Export.image.toAsset(**{
    'image': finalimage,
    'description': description,
    'assetId': 'projects/ee-webbe/assets/ALPOD/S2_classified/' + description,
    #'folder': directory,
    #'fileFormat': 'GeoTIFF',
    'scale': 10,
    'region': roi,
    'maxPixels': 1e12
})
task.start()
import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Streaming output truncated to the last 5000 lines.
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (id: HSNUFQXGF56V2VF7EGSALIGB).
Polling for task (